In [9]:
from bs4 import BeautifulSoup
import requests
import html5lib
import re
import pandas as pd

In [55]:
def clean_publish_year(raw_year):
    result = re.search(r".+\W+published\s*(\b\d+)", raw_year)
    # print(result.groups())
    # print(raw_year)
    if result == None:
        return "None"
    else:
        return result.group(1)

def clean_rating(raw_rating):
    result = re.search(r"(\b\d+.\b\d+)", raw_rating)
    if result == None:
        return "None"
    else:
        return result.group(1)

def clean_raters(raw_raters):
    result = re.search(r"((?:\d+,?\d+)*) ratings", raw_raters)
    if result == None:
        return "None"
    else:
        return result.group(1)

def clean_readers(raw_readers):
    result = re.search(r"(\b\d+)", raw_readers)
    if result == None:
        return "None"
    else:
        return result.group(1)


In [56]:
from numpy import int8


books = []
response = requests.get("https://www.goodreads.com/book/most_read")
html_soup = BeautifulSoup(response.text, 'html5lib')
book_container = html_soup.find_all('tr')
for book in book_container:
    bookTitle = book.find('a', class_ = "bookTitle").text.strip()
    bookAuthor = book.find('a', class_ = "authorName").text.strip()
    year = clean_publish_year(book.find('span', class_ = "greyText smallText uitext").text.strip())
    rating = clean_rating(book.find('span', class_ = "minirating").text.strip())
    raters = clean_raters(book.find('span', class_ = "minirating").text.strip()).replace(",", "")
    readerNum = clean_readers(book.find('span', class_ = "greyText statistic").text.strip())
    book_data = [bookTitle, bookAuthor, year, rating, raters, readerNum]
    books.append(book_data)
books = pd.DataFrame(books, columns=["Title", "Author", "Year", "Rating","Raters", "Readers"])
books.head()

,Title,Author,Year,Rating,Raters,Readers
0,Laut Bercerita,Leila S. Chudori,2017,4.64,10084,43
1,Jakarta Sebelum Pagi,Ziggy Zezsyazeoviennazabrizkie,2016,4.30,2662,40
2,The Seven Husbands of Evelyn Hugo,Taylor Jenkins Reid,2017,4.47,1529817,40
3,It Ends with Us,Colleen Hoover,2016,4.38,1651606,31
4,The Midnight Library,Matt Haig,2020,4.03,1125616,29


In [57]:
books.to_csv("Indo7.csv", index = False)